In [9]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [57]:
path = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/Table titles raw data/final_table_titles2.csv'
df = pd.read_csv(path, usecols = ['page_number','final_table_title', 'Application title short', 'DataID_pdf', \
                                  'categories', 'Category'])
df = df[df['categories'] > 0] 
df = df[df['Category'] == 'Table']
df['final_table_title'] = df['final_table_title'].str.title()
df.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
2,8,Table 1 Hydrometric Data From Discharge Statio...,0.75,1059614.pdf,Application for North Montney Project,Table
3,9,Table 2 Fish Species That May Occur In The Upp...,0.75,1059614.pdf,Application for North Montney Project,Table
4,14,Table 3 Summary Of Aquatics Field Work And Abo...,0.75,1059614.pdf,Application for North Montney Project,Table
5,17,Table 4 Summary Of Watercourse Crossings Along...,0.75,1059614.pdf,Application for North Montney Project,Table
6,18,Table 4 Summary Of Watercourse Crossings Along...,0.35,1059614.pdf,Application for North Montney Project,Table


In [64]:
df['Application title short'].unique().tolist()

['Application for North Montney Project',
 'Application for Trans Mountain Expansion Project',
 'Application for the Wolverine River Lateral Loop Carmon Creek Section',
 'Applications for Energy East, Asset Transfer and Eastern Mainline  Eastern Mainline ESA',
 'Application for the Line 3 Replacement Program',
 'Application to Construct and Operate Ekwan Pipeline',
 'Application for the 2017 NGTL System Expansion',
 'Application for the Towerbirch Expansion Project',
 'Application for the Albersun Pipeline Asset Purchase G',
 'Applications for Energy East, Asset Transfer and Eastern Mainline',
 'Application for the Wyndwood Pipeline Expansion Project',
 'Application for the Spruce Ridge Program',
 'Application for the construction of the West Path Delivery Project',
 'Application for the Goldboro Gas Plant and 26" Gathering Pipeline Abandonment',
 'Application for 2021 NGTL System Expansion Project',
 'Application for Leave to Abandon Deep Panuke Pipeline',
 'Application for the Constr

## Application for 2021 NGTL System Expansion Project

In [67]:
hearing = 'Application for 2021 NGTL System Expansion Project' 
ngtl2021 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2021.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,3,Table 5.1–1: Physical And Meteorological Data ...,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
1,4,Table 5.1–2: Biophysical Regions In Project Area,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
2,6,Table 5.1–3: Uppermost Bedrock Formation,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
3,7,Table 5.1–4: Surficial Geology,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
4,10,Table 5.1–5: Stations Used To Provide Climate ...,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table


## Application for the 2017 NGTL System Expansion

In [68]:
hearing = 'Application for the 2017 NGTL System Expansion'
ngtl2017 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2017.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,6,Table 8C.1 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
1,7,Table 8C.2 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
2,8,Table 8C.2 Cumulative Effects Characterization...,0.35,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
3,9,Table 8C.3 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
4,10,Table 8C.3 Cumulative Effects Characterization...,0.35,2748265.pdf,Application for the 2017 NGTL System Expansion,Table


## Application for Northwest Mainline Komie North Extension

In [69]:
hearing = 'Application for Northwest Mainline Komie North Extension'
komie_north = df[df['Application title short'] == hearing].reset_index(drop = True)
komie_north.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,7,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
1,9,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
2,11,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
3,13,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
4,15,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
